In [1]:
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain import VectorDBQA
from langchain import ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import DeepInfra
import os

llm = DeepInfra(model_id="meta-llama/Meta-Llama-3-70B-Instruct")
llm.model_kwargs = {
    "temperature": 0,
    "repetition_penalty": 1.1,
    "max_new_tokens": 25,
    "top_p": 0.9,
}

# Step 1: Load data
data_path = 'quant_boards.txt'
loader = TextLoader(data_path)
documents = loader.load()

# Step 2: Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [2]:
texts

[Document(page_content='Product 1: Quant 1\nРазмер: маленький\nЦена: $3 299\n**Описание:**\nQuant 1 — это идеальный выбор для начинающих и любителей серфинга, которые хотят ощутить магию квантовых технологий на воде. Компактный размер доски идеально подходит для тех, кто ищет маневренность и легкость в управлении.\n**Мощность двигателя:**', metadata={'source': 'quant_boards.txt'}),
 Document(page_content='**Мощность двигателя:**\nОснащен квантовым двигателем мощностью 500 кВт, который обеспечивает плавное и стабильное движение на воде, независимо от условий. Этот двигатель позволяет новичкам почувствовать уверенность и комфорт при катании.\n**Кому подойдет:**\nQuant 1 создан для новичков и тех, кто предпочитает легкость и маневренность. Это отличный выбор для молодых серферов и тех, кто хочет освоить базовые навыки серфинга.\nProduct 2: Quant 2\nРазмер: большой\nЦена: $7 659\n**Описание:**', metadata={'source': 'quant_boards.txt'}),
 Document(page_content='Размер: большой\nЦена: $7 659

In [8]:
from sentence_transformers import SentenceTransformer
from chromadb import Client
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import chromadb
# Step 1: Load data
data_path = 'quant_boards.txt'
loader = TextLoader(data_path)
documents = loader.load()

# Step 2: Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Step 3: Create embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # Вы можете выбрать другую модель по вашему желанию
embeddings = [model.encode(text.page_content) for text in texts]



c:\Users\ataka\anaconda3\envs\gspace\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
client = chromadb.PersistentClient(path="./chromadb")


In [10]:
# # Step 4: Initialize Chroma DB and add embeddings
# client = Client(Settings(chroma_db_path="./chroma_db"))
collection = client.create_collection("my_collection")



In [16]:
for i, (text, embedding) in enumerate(zip(texts, embeddings)):
    collection.add(
        ids=str(i),
        embeddings=embedding.tolist(),
        metadatas={"text": text.page_content}
    )

In [19]:
from langchain import ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import DeepInfra
from sentence_transformers import SentenceTransformer
from chromadb import Client
from chromadb.config import Settings
import os


# Инициализация LLM
llm = DeepInfra(model_id="mistralai/Mistral-7B-Instruct-v0.3")
llm.model_kwargs = {
    "temperature": 0.7,
    "repetition_penalty": 1,
    "max_new_tokens": 250,
    "top_p": 0.9,
    "top_k": 0
}

# Инициализация модели для создания эмбеддингов
model = SentenceTransformer('all-MiniLM-L6-v2')

# Инициализация Chroma DB
client = chromadb.PersistentClient(path="./chromadb")
collection = client.get_collection("my_collection")



c:\Users\ataka\anaconda3\envs\gspace\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [44]:
# Определение шаблона запроса
template = """
    Никогда не забывайте, что ваше имя Гоша. 
    Вы работаете специалистом-консультантом по подбору серфигбордов на квантовых двигателях. 

    {human_input}
    """

prompt = PromptTemplate(
    input_variables=["history", "retrieved_info", "human_input"], 
    template=template
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=10),
)

def retrieve_relevant_documents(query, top_k=3):
    query_embedding = model.encode([query])[0]
    results = collection.query(query_embedding.tolist(), n_results=top_k)

    retrieved_texts = [result['text'] for result in results['metadatas'][0]]
    return "\n".join(retrieved_texts)

def respond_to_user(user_input):
    retrieved_info = retrieve_relevant_documents(user_input)
    history = llm_chain.memory.load_memory_variables({})["history"]
    query_rag_template = f'''
При ответе на сообщение используй следующую информацию:
{retrieved_info}
    
История диалога:
{history}

Текущее сообщение собеседника:
{user_input}

'''
    response = llm_chain.predict(human_input=query_rag_template)
    return response


In [45]:
# Пример использования
user_input = "Привет"
response = respond_to_user(user_input)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

    Никогда не забывайте, что ваше имя Гоша. 
    Вы работаете специалистом-консультантом по подбору серфигбордов на квантовых двигателях. 

    
При ответе на сообщение используй следующую информацию:
Эти серфингборды с квантовыми двигателями предложат незабываемые впечатления на воде и помогут каждому серферу найти свой идеальный стиль катания!
**Мощность двигателя:**
Оснащен квантовым двигателем мощностью 500 кВт, который обеспечивает плавное и стабильное движение на воде, независимо от условий. Этот двигатель позволяет новичкам почувствовать уверенность и комфорт при катании.
**Кому подойдет:**
Quant 1 создан для новичков и тех, кто предпочитает легкость и маневренность. Это отличный выбор для молодых серферов и тех, кто хочет освоить базовые навыки серфинга.
Product 2: Quant 2
Размер: большой
Цена: $7 659
**Описание:**
Размер: большой
Цена: $7 659
**Описание:**
Quant 2 — это серфингборд для профессионалов и продвинутых п